In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformer_lens import HookedTransformer 
import torch
from transformer_lens.hook_points import (
    HookPoint,
)  # Hooking utilities
from jaxtyping import Float
import tqdm
import transformer_lens.utils as utils
from functools import partial
from tabulate import tabulate

In [2]:
#Loading model - let's use Pythia 
#EleutherAI/pythia-410m-deduped
hooked_model = HookedTransformer.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")

Loaded pretrained model gpt2 into HookedTransformer


In [43]:
tokens = tokenizer.encode("I believe in God ", return_tensors="pt")
with torch.no_grad():
    logits, clean_cache = hooked_model.run_with_cache(tokens)

In [44]:
def residual_stream_patching_hook(
    resid_post: Float[torch.Tensor, "batch pos d_model"],
    hook: HookPoint,
    position: int
) -> Float[torch.Tensor, "batch pos d_model"]:
    clean_resid_post = clean_cache[hook.name]
    resid_post[:, 0, :] = clean_resid_post[:, position, :]
    return resid_post

num_positions = len(tokens[0])
patching_result = torch.zeros((hooked_model.cfg.n_layers, num_positions, 50), device=hooked_model.cfg.device)

for layer in tqdm.tqdm(range(hooked_model.cfg.n_layers)):
    for position in range(num_positions):
        # Use functools.partial to create a temporary hook function with the position fixed
        temp_hook_fn = partial(residual_stream_patching_hook, position=position)
        # Run the model with the patching hook
        patched_logits = hooked_model.run_with_hooks(torch.tensor([[0]], device='cuda:0'), fwd_hooks=[
            (utils.get_act_name("resid_post", layer), temp_hook_fn)
        ])
        # Calculate the logit difference      
        _, patching_result[layer, position] = torch.topk(patched_logits.squeeze(), k=50)

100%|██████████| 12/12 [00:03<00:00,  3.09it/s]


In [45]:
def stringify(x):
    return hooked_model.to_string(x)
result = [[[stringify(top) for top in element]for element in subarray] for subarray in patching_result.int()]
strtoks = [stringify(tok) for tok in tokens[0]]

In [47]:
_ , clean_predicts = torch.topk(logits.squeeze(), k=5)
clean_predicts_text = [[stringify(tok) for tok in elem] for elem in clean_predicts.int()]
result.append(clean_predicts_text)

In [48]:
print(tabulate(result, headers=strtoks, tablefmt="fancy_grid"))

╒═══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╤═══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╤═══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════

## Comparing TunedLens and ourLens over IOI task

In [11]:
import torch
from tuned_lens.nn.lenses import TunedLens, LogitLens
from transformers import AutoModelForCausalLM, AutoTokenizer

device = torch.device('cpu')
# To try a diffrent modle / lens check if the lens is avalible then modify this code
model = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tuned_lens = TunedLens.from_model_and_pretrained(model)
tuned_lens = tuned_lens.to(device)
logit_lens = LogitLens.from_model(model)

In [12]:
from tuned_lens.plotting import PredictionTrajectory
import ipywidgets as widgets
from plotly import graph_objects as go


def make_plot(lens, text, layer_stride, statistic, token_range):
    input_ids = tokenizer.encode(text)
    targets = input_ids[1:] + [tokenizer.eos_token_id]

    if len(input_ids) == 0:
        return widgets.Text("Please enter some text.")
    
    if (token_range[0] == token_range[1]):
        return widgets.Text("Please provide valid token range.")
    pred_traj = PredictionTrajectory.from_lens_and_model(
        lens=lens,
        model=model,
        input_ids=input_ids,
        tokenizer=tokenizer,
        targets=targets,
    ).slice_sequence(slice(*token_range))

    return getattr(pred_traj, statistic)().stride(layer_stride).figure(
        title=f"{lens.__class__.__name__} ({model.name_or_path}) {statistic}",
    )

style = {'description_width': 'initial'}
statistic_wdg = widgets.Dropdown(
    options=[
        ('Entropy', 'entropy'),
        ('Cross Entropy', 'cross_entropy'),
        ('Forward KL', 'forward_kl'),
    ],
    description='Select Statistic:',
    style=style,
)
text_wdg = widgets.Textarea(
    description="Input Text",
    value="it was the best of times, it was the worst of times",
)
lens_wdg = widgets.Dropdown(
    options=[('Tuned Lens', tuned_lens), ('Logit Lens', logit_lens)],
    description='Select Lens:',
    style=style,
)

layer_stride_wdg = widgets.BoundedIntText(
    value=2,
    min=1,
    max=10,
    step=1,
    description='Layer Stride:',
    disabled=False
)

token_range_wdg = widgets.IntRangeSlider(
    description='Token Range',
    min=0,
    max=1,
    step=1,
    style=style,
)


def update_token_range(*args):
    token_range_wdg.max = len(tokenizer.encode(text_wdg.value))

update_token_range()

token_range_wdg.value = [0, token_range_wdg.max]
text_wdg.observe(update_token_range, 'value')

interact = widgets.interact.options(manual_name='Run Lens', manual=True)

plot = interact(
    make_plot,
    text=text_wdg,
    statistic=statistic_wdg,
    lens=lens_wdg,
    layer_stride=layer_stride_wdg,
    token_range=token_range_wdg,
)

interactive(children=(Dropdown(description='Select Lens:', options=(('Tuned Lens', TunedLens(
  (unembed): Une…

## Some previous generations

see how there's when John and Mary went to church which is due to the association in the embeddings (I think)

generations on hook_resid_pre

In [83]:
print(tabulate(result, headers=strtoks, tablefmt="fancy_grid"))

╒════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╤═════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╤════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════

In [78]:
print(tabulate(result, headers=strtoks, tablefmt="fancy_grid"))

╒════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╤═════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╤════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════

layer 8: first layer where there's Mary in the top 50 but after John, layer 9 molto più avanti
In the case in which you would need to have John predicted there's not John in the top 50 of layer 8 but is present in layer 9


In [65]:
print(tabulate(result, headers=strtoks, tablefmt="fancy_grid"))

╒═══════════════════════════════════════════════════════════════════════╤══════════════════════════════════════════════════════════════════════════════════════════════╤═════════════════════════════════════════════════════════════════╤══════════════════════════════════════════════════════════════════════╤═══════════════════════════════════════════════════════════════════════════════╤══════════════════════════════════════════════════════════════════════════════════════════════════╤═════════════════════════════════════════════════════════════════════════════════════════════════════════╤═════════════════════════════════════════════════════════════════════════════════════════════════════════════╤═════════════════════════════════════════════════════════════════════════════════════════════════════════╤═══════════════════════════════════════════════════════════════════════════════════════════════╕
│ The                                                                   │  Tour                  

In [55]:
print(tabulate(result, headers=strtoks, tablefmt="fancy_grid"))

╒════════════════════════════════════════════════════════════════════╤════════════════════════════════════════════════════════════════════════════════════════════════╤═══════════════════════════════════════════════════════════════════════════════════════════╤══════════════════════════════════════════════════════════════════════════════════════════════════════════════╤══════════════════════════════════════════════════════════════════════════════════════════════════════╤════════════════════════════════════════════════════════════════════════════════╤═════════════════════════════════════════════════════════════════════════════════════════════════════════════╤═════════════════════════════════════════════════════════════════════════════════╤═══════════════════════════════════════════════════════════════════════════════════════════╤════════════════════════════════════════════════════════════════════════════════════════════╤═════════════════════════════════════════════════════════════════════

In [50]:
print(tabulate(result, headers=strtoks, tablefmt="fancy_grid"))

╒════════════════════════════════════════════════════════════════════════╤════════════════════════════════════════════════════════════════════════════════════════════════════╤══════════════════════════════════════════════════════════════════════════════════════════════╤═══════════════════════════════════════════════════════════════════════════════════════════════════════════╤══════════════════════════════════════════════════════════════════════════════════════╤═════════════════════════════════════════════════════════════════════════════════════════════╤══════════════════════════════════════════════════════════════════════════════════════════════════════╤════════════════════════════════════════════════════════════════════════════════════════════════════════╤═════════════════════════════════════════════════════════════════════════════════════╤═══════════════════════════════════════════════════════════════════════════════════════════════════════════╤═══════════════════════════════════════

Trying Llama2